In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
219,AFG,Asia,Afghanistan,2020-08-16,37551.0,120.0,1370.0,7.0,964.619,3.083,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
380,ALB,Europe,Albania,2020-08-16,7260.0,143.0,225.0,6.0,2522.760,49.691,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
605,DZA,Africa,Algeria,2020-08-16,38133.0,469.0,1360.0,9.0,869.603,10.695,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-15,France,214294.0,0.0,30443.0,0.0,0.0,0.0
0,2020-08-15,China,89287.0,0.0,4708.0,0.0,0.0,0.0
0,2020-08-15,Italy,253376.0,0.0,35261.0,0.0,0.0,0.0
0,2020-08-15,Spain,348889.0,0.0,28646.0,0.0,0.0,0.0
0,2020-08-15,United States,5393486.0,0.0,169573.0,0.0,0.0,0.0
0,2020-08-15,World,21607231.0,0.0,767638.0,0.0,0.0,0.0
0,2020-08-15,United Kingdom,317579.0,0.0,37130.0,0.0,0.0,0.0
0,2020-08-15,Germany,224131.0,0.0,9249.0,0.0,0.0,0.0
0,2020-08-15,Iran,342586.0,0.0,19572.0,0.0,0.0,0.0
0,2020-08-15,Turkey,247953.0,0.0,5953.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-16,France,214294.0,215521.0,30443.0,30409.0,-1227.0,34.0
0,2020-08-16,China,89287.0,89279.0,4708.0,4703.0,8.0,5.0
0,2020-08-16,Italy,253376.0,253438.0,35261.0,35392.0,-62.0,-131.0
0,2020-08-15,Spain,348889.0,342813.0,28646.0,28617.0,6076.0,29.0
0,2020-08-16,United States,5393486.0,5361165.0,169573.0,169481.0,32321.0,92.0
0,2020-08-16,World,21607231.0,21472570.0,767638.0,766080.0,134661.0,1558.0
0,2020-08-16,United Kingdom,317579.0,316367.0,37130.0,41358.0,1212.0,-4228.0
0,2020-08-16,Germany,224131.0,223453.0,9249.0,9231.0,678.0,18.0
0,2020-08-16,Iran,342586.0,341070.0,19572.0,19492.0,1516.0,80.0
0,2020-08-16,Turkey,247953.0,248117.0,5953.0,5955.0,-164.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")